In [2]:
# call %matplotlib only in a Jupiter notebook
%matplotlib inline 

# Common and local imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

# Overview

This notebook contains the code to train and save the initial twenty convolutional layers that will be pre-trained for optimization purposes.

The full model contains a total of twenty four convolutional layers. The initial twenty (what we create here) will be tasked on feature extraction and 

# Reading inputs
To-Do

In [4]:
# Create synset to words map of all synsets currently 
# supported in ImageNet
synset_to_word = dict([])

words = open('words.txt','r')

for line in words:
    if line.strip():
        line = line.rstrip().split("\t")
        synsets_str = line[1]
        synsets = [synsets_str]
        if "," in synsets_str:
            synsets = synsets_str.split(",")

        synset_to_word[line[0]] = synsets

words.close() # always close your files.


# constants
LAYER_TYPES = dict([
    ("conv", "convolutional"),
    ("conn", "connected"),
    ("det", "detection"),
    ("drop", "dropout"),
    ("local", "local"),
    ("maxpool", "maxpool")
])

classes = ['dogs', 'cats'] 
num_classes = len(classes)
train_path='training_data'


# Input values
img_size = 448 # 448 x 448 RGB image
num_channels = 3 # JPEG images have 3 channels (RGB)
batch_size = 64 # batch size

# Image configurations 
subdiv=7 # subdivisions for the image
momentum=0.9
decay=0.0005
saturation=1.5
exposure=1.5
hue=.1

# Learning Rate and Network graph params
learning_rate=0.0005
steps=200,400,600,20000,30000
policy=steps
scales=2.5,2,2,.1,.1
max_batches = 40000

# data = dataset.read_train_sets(train_path, img_size, classes, validation_size=validation_size)

# Utility functions

In [31]:
def create_biases(size):
    """
    To-Do
    """
    return tf.Variable(tf.constant(0.05, shape=[size]))

def create_weights(shape, stddev=0.05):
    """
    To-Do
    """
    return tf.Variable(tf.truncated_normal(shape, stddev=stddev))

def create_flatten_layer(layer):
    """
    To-Do
    """
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])
    
    return layer

def create_fully_connected_layer(input,          
                    num_inputs,    
                    num_outputs,
                    use_relu=True):
    """
    To-Do
    """
    #Let's define trainable weights and biases.
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)
 
    layer = tf.matmul(input, weights) + biases
    
    if use_relu:
        layer = tf.nn.relu(layer)
 
    return layer

def create_convolutional_layer(input,
                               num_input_channels,
                               conv_filter_size,
                               num_filters, 
                               subdivisions, 
                               use_relu=True):
    """
    This function facilitates the creation of a convolution layer for the YOLO algorithim.
    
    
    Args:
        input (Tensor): the output (activation) from the previous layer. This should be a 4-D tensor. 
            Typically, in the first convolutional layer, you pass n images of size width * height * num_channels, 
            then this has the size [n width height num_channels].

        num_input_channels (int): the number of imput channels for the image. JPG's have RGB channels, so it 
            would be 3 while PNG's have an extra channel called alpha (RGBA) so in this case it would be  4.
        
        conv_filter_size (int): Size that will be use for the width and height of the filter.
        
        num_filters (int): The number of filters that this layer will contain.
        
        subdivisions (int): The dimension for the image segmentation size. The number of grids per row in the image
        
        use_relu (bool): By default it's true. Whether to use relu before returning layer or not.
        
    Return:
        Tensor: returns the activation function output if use_relu = True, otherwise, return the output of 
            the maxpooling output.
    """
    
    # We shall define the weights that will be 
    # trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, 
                                    conv_filter_size, 
                                    num_input_channels, 
                                    num_filters])
    
    # We create biases using the create_biases function. 
    # These are also trained.
    biases = create_biases(num_filters)

    # Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    layer += biases

    # We shall be using max_pool (max pooling).  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    
    if(use_relu):    
        # Output of pooling is fed to Relu which 
        # is the default activation function for us.
        layer = tf.nn.relu(layer)

    return layer

def create_network(net_description = []):
    """
    To-Do
    
    Example:
        create_network([
            net_description_item(batch_normalize=1,
                filters=1024,
                layer_type: LAYER_TYPES['conv'],
                pad=1,
                size=3,
                stride=1,
                activation=leaky),    
            net_description_item(layer_type: LAYER_TYPES["maxpool"],
                size=2,
                stride=2),
            .
            .
            .
            net_description_item(layer_type: LAYER_TYPES["conn"],
                output=1715
                activation=linear),
        ])
    
    
    Args:
        net_description ( Array(Object) ): 
    """
    with tf.device('/device:GPU:0'):
        
        input = tf.placeholder(tf.float32, 
                   shape=[None, img_size,img_size, num_channels], 
                   name='x')
        
        network = []
        
        for layer in net_description:
            
            layer_type = layer["layer_type"]
            
            if LAYER_TYPES["conv"] in layer_type:
                network.append(create_convolutional_layer(input,
                                                     num_input_channels=num_channels,
                                                     conv_filter_size=filter_size_conv1,
                                                     num_filters=num_filters_conv1, 
                                                     subdivisions=subdiv, 
                                                     use_relu=False)
                              )
                input = network[-1]
                
            elif LAYER_TYPES["conn"] in layer_type:
    
            elif LAYER_TYPES["det"] in layer_type:

            elif LAYER_TYPES["drop"] in layer_type:

            elif LAYER_TYPES["local"] in layer_type:

            else: # maxpool
        
    return network

        
def net_description_item(activation="leaky", batch_normalize=1, classes=num_classes, coords=4, 
                         filters=1024, jitter=.2, layer_type='convolutional', num=3, output=1715, 
                         pad=1, probability=.5, rescore=1, softmax=0, sqrt=1, side=7, size=3, 
                         stride=1):
    """
    To-Do
    
    Helper for the create_network function. Helps in creation of network 
    description dictionary items
    
    Args:
        activation (str): ['leaky' | 'linear']
    """
    net_item = dict([("layer_type", layer_type)])
    
    if LAYER_TYPES["conv"] in layer_type:
        net_item["activation"] = activation
        net_item["batch_normalize"] = batch_normalize
        net_item["filters"] = filters
        net_item["size"] = size
        net_item["stride"] = stride
        net_item["pad"] = pad
        
    elif LAYER_TYPES["conn"] in layer_type:
        net_item["activation"] = activation
        net_item["output"] = output
    
    elif LAYER_TYPES["det"] in layer_type:
        net_item["classes"] = classes
        net_item["coords"] = coords
        net_item["rescore"] = rescore
        net_item["size"] = size
        net_item["num"] = num
        net_item["softmax"] = softmax
        net_item["sqrt"] = sqrt
        net_item["jitter"] = jitter
        
    elif LAYER_TYPES["drop"] in layer_type:
        net_item["probability"] = probability
        
    elif LAYER_TYPES["local"] in layer_type:
        net_item["activation"] = activation
        net_item["filters"] = filters
        net_item["size"] = size
        net_item["stride"] = stride
        net_item["pad"] = pad
    
    else: # maxpool
        net_item["size"] = size
        net_item["stride"] = stride
        
    return net_item


def train(num_iteration):
    """
    To-Do
    """
    total_iterations = 0
    loss = []
    epochs = []
    accuracy_array = []
    
    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:   
        session.run(tf.global_variables_initializer())
        
        for i in range(total_iterations,
                       total_iterations + num_iteration):

            x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size)
            x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(batch_size)
            
            feed_dict_tr = {
                x: x_batch, 
                y_true: y_true_batch
            }
        
            feed_dict_val = {
                x: x_valid_batch,
                y_true: y_valid_batch
            }

            session.run(optimizer, feed_dict=feed_dict_tr)

            if i % int(data.train.num_examples/batch_size) == 0: 
                val_loss = session.run(cost, feed_dict=feed_dict_val)
                epoch = int(i / int(data.train.num_examples/batch_size))    

                loss.append(val_loss)
                epochs.append(epoch)
                accuracy_array.append(session.run(accuracy, feed_dict=feed_dict_tr))
                
                loss_map = {
                    'loss': loss,
                    'epochs': epochs,
                    'accu': accuracy_array 
                }

#                 if (epoch > 0) and (epoch % 5) == 0:
#                     pd_loss = pd.DataFrame(loss_map)
#                     pd_loss.plot(x="accu", y="loss", kind='line')
#                     plt.show()  
                
                show_progress(epoch, 
                              feed_dict_tr, 
                              feed_dict_val,
                              val_loss,
                              session)

                saver.save(session, './dogs-cats-model') 
                
        total_iterations += num_iteration

IndentationError: expected an indented block (<ipython-input-31-1789c8a238c0>, line 141)